## Linear Algebra: Solving equations
__MATH 420__ <br>
_Spring 2021_ <br>


Let's solve a $3 \times  3$ linear  system of equations. The coefficient matrix is a Julia array; to enter it by hand, we write the members of each row separated with spaces (_not_ commas) and separate each row with a semicolon. Our coefficient matrix is

In [ ]:
Mat = [1 2 3; 4 5 6; 7 8 19]

Similarly we can enter the right hand side (RHS) by hand too. We'll assign `b` to the RHS

In [ ]:
b = [2; 5; 10]

We can solve the linear equations `Mat x = b` with the backslash command:

In [ ]:
x  = Mat \ b

The result is a column vector of the solution.

Let's check our work.  In Julia,the  asterisk also means matrix multiplication.  Let's find `b - Mat * x`. This difference is called the _residual_.  For the true solution, the residual is the zero vector; let's check:

In [ ]:
b - Mat * x 

Super, the residual is the zero vector. Almost surely, the solution is correct.

If we need to do exact rational arithmetic, we can do that. Actually, instead of using the default of `Int64` numbers, let's convert to `BigInt` numbers:

In [ ]:
Mat = map(x -> convert(BigInt, x)//1,Mat)

In [ ]:
b = map(x -> convert(BigInt, x) //1,b)

In [ ]:
Mat \ b

For larger or more complex coefficient matrices, we can define a function that evaluates so the $i,j$ matrix element; a famous example:

In [ ]:
function F(i,j) 
   if i == j  
      2
    elseif abs(i - j) == 1
      1
    elseif abs(i-j) == 2
      -2
    else 
       0
  end
end

In [ ]:
n = 9;

An array comprehension generates our array:

In [ ]:
M = [F(i,j) for i=1:n, j=1:n]

The matrix is a _tridiagonal matrix_. Let's solve $M x = b$, where $b$ is a column vector with each member equal to $1$

In [ ]:
b = [1  for i=1:n]

In [ ]:
x  = M  \ b

Checking our work, our solution looks OK-- some members of the residual are about equal to  the machine epsilon--not surprising.  One member is about ten times the machine epsilon

In [ ]:
M * x - b

Julia has a function that computes the matrix inverse. But solving using the matrix inverse is _slower_ than using the backslash; we need to run the tests twice! For our test, let's use a $4096 \times 4096$ matrix

In [ ]:
n = 2^12

In [ ]:
M = [F(i,j) for i=1:n, j=1:n];

In [ ]:
x = [1  for i=1:n];

In [ ]:
b = M * x;

In [ ]:
@time M \ b;

In [ ]:
@time x = M \ b;

In [ ]:
@time inv(M) * b;

In [ ]:
@time xx = inv(M) * b;

Solving using the matrix inverse is slower and uses more memory--using a matrix inverse is almost always the wrong thing to do. 

Are the two solutions nearly equal? Let's hope so.  The Julia function `findmax` will find the maximum member of an array and also find its position.  We might like the member with the greatest magnitude, so we'll map the absolute value function onto the difference:

In [ ]:
findmax(map(abs, x - xx))

In [ ]:
Mat = [1 2 3; 4 5 6; 7 8 107]

In [ ]:
b = [1 0 0; 0 1 0 ; 0 0 1]

Each column is a solution. The first column is the solution with the RHS [1,0,1]

In [ ]:
x = Mat \ b

Mat * x should be the $3 \times 3 $  identity matrix; let's check:

In [ ]:
Mat * x

Is it the $3 \times 3 $  identity matrix? Well, almost--the off-diagonal terms differ from zero by an amount that is about the same or smaller than the machine epsilon. We're not surprised by this.  

Let's try another example. A famous matrix that has a determinant that is zero is 
$$
 \begin{bmatrix} 1&2&3\\4&5& 6 \\ 7 & 8 & 9 \end{bmatrix}
$$
Let's change the $3,3$ element just a bit and use it as a coefficient matrix. For just a bit, we'll use $2^{-46}$.

In [ ]:
Mat = [1 2 3; 4 5 6; 7 8 9+ 2.0^(-46)]

In [ ]:
b = [1 0 0; 0 1 0 ; 0 0 1]

As a matrix element, $9+ 2^{-46}$ prints as $9.0$, but it's value isn't 9:

In [ ]:
9+ 2.0^(-46)

In [ ]:
x = Mat \ b

Should we check our work? Always!

In [ ]:
Mat * x

Yikes! This should be the $3 \times 3$ identity matrix, but the off diagonal terms differ a _great deal_ from zero. What's the story? Let's try solving with exact rational numbers

In [ ]:
MMat = map(x -> BigInt(x)//1, [1 2 3; 4 5 6; 7 8 9]) + [0 0 0; 0 0 0; 0 0 1//BigInt(2)^46]

In [ ]:
b = map(x -> BigInt(x)//1, b)

In [ ]:
xx = MMat \ b

Checking this, all is well!

In [ ]:
MMat * xx

Let's compare the two solutions

In [ ]:
xx = map(x -> convert(Float64,x), xx)

In [ ]:
findmax(map(abs, x - xx))

The absolute values of the differences is as large as about $0.046875.$

What we are seeing is something like the butterfly effect. A tiny change in the input to a linear system, can cause a big change in the output. Whence the changes in the input? We need to round the matrix elements to floating point numbers. Plus every arithmetic operation involves more rounding errors.

The matrix condition number (which we will learn about) is a measure of how much the solution to a linear system might change due to a change in the inputs. Julia gives us a quick way to compute the matrix condition number>

In [ ]:
using LinearAlgebra

In [ ]:
cond(Mat,Inf)

I know what you are thinking: The condition number of our matrix is big because its determinate is small:

In [ ]:
det(Mat)

In [ ]:
MM = [6.086956521743044e5 9.130434782599565e5; 2.608695652171305e5 3.913043478266957e5]

In [ ]:
det(MM)

In [ ]:
cond(MM, Inf)

Returning to our $4096 \times 4096$ matrix, let's find its condition number

In [ ]:
cond([F(i,j) for i=1:4096, j=1:4096], Inf)

Again, we'll learn more about what the matrix condition number means